## Task 1. N-gram models

Create different pipelines for extracting features from texts using 1-gram, 2-gram and 3-gram symbol counting model. Test your pipeline by extracting features from a subset of the 20 newsgroup dataset (https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html). Evaluate performance of your pipelines. How arity of the the gram model affects performance?

## Task 2. Custom transformers
Create your custom transformer, which filters all N-grams from the text, which has the the highest frequency.

## Task 3. Extracting features from images
Create a pipeline for extracting features from images. Try to make your pipeline as versitile as possible, so that you can create different feature models. Consider following options:

1. Informativeness of the model (how many information features contain).
2. Complexity of the model (how hard it is to extract features)
3. Preprocessing of features (is it necessary to perform some kind of additional transformation on the extracted features).

In the end you must have a number of configured transformers and feature extractors, which you have assembled in different pipelines.

Test your pipeline with the MNIST Database of hand-written digits: https://archive.ics.uci.edu/dataset/683/mnist+database+of+handwritten+digits or https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html#sklearn.datasets.load_digits.

Is it possible to somehow evaluate or compare informativeness of the models without solving classification or regression tasks. 